In [ ]:
import requests
import pandas as pd
from time import sleep
from datetime import datetime

In [ ]:
# Define the base URL for the API
base_url = "https://data.cityofnewyork.us/resource/8h9b-rp9u.json"

# Define headers including the app token
headers = {
    'X-App-Token': 'gcDEQgxy8Z5tdm905ovo9OhrQ'
}

# Function to handle pagination, fetch all data, and implement a delay between requests
def fetch_all_data(base_url, headers, start_date, end_date):
    limit = 50000  # Adjust the limit as needed, depending on what the API allows
    offset = 0
    full_data = []

    # Format start and end dates for the API call
    formatted_start_date = datetime.strptime(start_date, "%Y-%m-%d").strftime('%Y-%m-%dT%H:%M:%S.%f')
    formatted_end_date = datetime.strptime(end_date, "%Y-%m-%d").strftime('%Y-%m-%dT%H:%M:%S.%f')

    while True:
        # Construct the query with pagination and date filtering
        query_url = f"{base_url}?$limit={limit}&$offset={offset}" \
                    f"&$where=ARREST_DATE between '{formatted_start_date}' and '{formatted_end_date}'"
        
        response = requests.get(query_url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to fetch data: HTTP Status Code {response.status_code}")
            break
        
        data = response.json()
        if not data:
            break  # Break the loop if no more data is returned
        
        full_data.extend(data)
        offset += limit
        
        # Sleep for 5 seconds before making the next request
        sleep(5)
    
    return full_data

# Set your desired date range
start_date = '2014-01-01'
end_date = '2023-12-31'

# Fetch all data using the defined function
all_data = fetch_all_data(base_url, headers, start_date, end_date)

# Create a DataFrame from the fetched data
historical_df = pd.DataFrame(all_data)


In [ ]:
print(historical_df.shape[0])

In [ ]:
# Converting the list of data into a DataFrame
historical_df = pd.DataFrame(all_data)
historical_df

In [ ]:
historical_df.dtypes

In [ ]:
arrest_counts = historical_df.groupby('arrest_date').size()
arrest_counts.plot()

In [ ]:
historical_df['ofns_desc'].value_counts()

In [ ]:
queens_data = historical_df[historical_df['arrest_boro'] == 'Q']
queens_crimes_per_year = queens_data.groupby('Year').size()
queens_crimes_per_year.plot()
